In [1]:
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.patheffects as mpe
import matplotlib
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("ENC_29 - Sheet14.csv")

In [4]:
df.columns

Index(['sr#', 'ID', 'ExpID', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'InterIntervalTime_gt100', 'AveragePTime',
       'AveragePTime_gt100', 'TP', 'Total_Packets', 'Total_Packets_gt100',
       'STD', 'Min', 'Max', 'Width_Height', 'P1203', 'Stall', 'Bitrate',
       'P1203C', 'P1203_2C', 'Stall_Binary'],
      dtype='object')

In [5]:
# df =df[(df['Algo']=='conventional') | (df['Algo']=='elastic')]

In [6]:
df.shape[0]

5049

In [7]:
df['P1203_2C'].unique()

array(['Bad', 'Fair', 'Good', 'Excellent'], dtype=object)

In [8]:
df['STD']= df['STD'].fillna(value=df['STD'].mean())

In [9]:
# # df.loc[df['Width_Height'] == '320x180', 'Width_Height'] = 1
# # cond16 = (df['daychange']<8) & (df['daychange']>2)
# def Classification(CLASS):
#     if CLASS == 's1-eth2':
#         return 1
#     elif CLASS == 's1-eth1':
#         return 'S1'
#     elif CLASS == 's2-eth2':
#         return 3
#     elif CLASS == 's2-eth1':
#         return 'S2'
#     elif CLASS == 's3-eth2':
#         return 5
#     elif CLASS == 's3-eth1':
#         return 6
#     elif CLASS == 's8-eth1':
#         return 7

In [10]:
# df['LR']=df['interface'].apply(Classification)

In [11]:
from sklearn import preprocessing


In [12]:
X= df.iloc[:, 6:14]

In [13]:
# X=preprocessing.normalize(X)
X= preprocessing.scale(X)

In [15]:
y = df.iloc[:, 22]

In [16]:
y

0       0
1       0
2       0
3       0
4       0
       ..
5044    0
5045    0
5046    0
5047    0
5048    0
Name: Stall_Binary, Length: 5049, dtype: int64

In [17]:
X.shape[0]

5049

In [18]:
sm = SMOTE()
X, y = sm.fit_sample(X, y)

In [19]:
X.shape[0]

8544

In [20]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify=y, random_state = 42)

In [21]:
# Fitting Classifier to the Training Set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion="entropy", max_depth=5)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [22]:
dt1_score_train = classifier.score(X_train, y_train)
print("Training score: ", dt1_score_train)
dt1_score_test = classifier.score(X_test, y_test)
print("Testing score: ", dt1_score_test)


Training score:  0.8045150501672241
Testing score:  0.7886115444617785


In [23]:
# Model performance on training set
y_pred_train =classifier.predict(X_train)

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train))

Accuracy: 0.80
Confusion Matrix: 
 [[2061  929]
 [ 240 2750]]
              precision    recall  f1-score   support

           0       0.90      0.69      0.78      2990
           1       0.75      0.92      0.82      2990

    accuracy                           0.80      5980
   macro avg       0.82      0.80      0.80      5980
weighted avg       0.82      0.80      0.80      5980



In [24]:
#Model Performance on Testing Data
y_pred_test =classifier.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred_test)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_test,y_pred_test)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred_test))

Accuracy: 0.79
Confusion Matrix: 
 [[ 842  440]
 [ 102 1180]]
              precision    recall  f1-score   support

           0       0.89      0.66      0.76      1282
           1       0.73      0.92      0.81      1282

    accuracy                           0.79      2564
   macro avg       0.81      0.79      0.78      2564
weighted avg       0.81      0.79      0.78      2564



In [25]:
print(classifier.tree_.max_depth)

5


In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6)

In [27]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=6)

In [28]:
knn.score(X_test,y_test)

0.8482839313572543

In [29]:
y_pred = knn.predict(X_test)

In [30]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8482839313572543


In [31]:
cm=confusion_matrix(y_test,y_pred)
print('Confusion Matrix: \n', cm)
print(classification_report(y_test, y_pred))

Confusion Matrix: 
 [[1069  213]
 [ 176 1106]]
              precision    recall  f1-score   support

           0       0.86      0.83      0.85      1282
           1       0.84      0.86      0.85      1282

    accuracy                           0.85      2564
   macro avg       0.85      0.85      0.85      2564
weighted avg       0.85      0.85      0.85      2564



In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=300)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300)

In [33]:
rf_predictions = rf.predict(X_test)
accuracy_score(y_test, rf_predictions)
print('Accuracy is ' + str(round(accuracy_score(y_test, rf_predictions),2)))

Accuracy is 0.9


In [34]:
print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1282
           1       0.89      0.90      0.90      1282

    accuracy                           0.90      2564
   macro avg       0.90      0.90      0.90      2564
weighted avg       0.90      0.90      0.90      2564



In [35]:
cm = confusion_matrix(y_test, rf_predictions)
print('Confusion matrix\n\n', cm)

Confusion matrix

 [[1139  143]
 [ 125 1157]]


In [36]:
print('Training set score: {:.4f}'.format(rf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(rf.score(X_test, y_test)))

Training set score: 1.0000
Test set score: 0.8955


In [37]:
rf

RandomForestClassifier(n_estimators=300)

In [38]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# encoder = LabelEncoder()
# encoder.fit(y)
# encoded_Y = encoder.transform(y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

In [39]:
dummy_y

NameError: name 'dummy_y' is not defined

In [42]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=8,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(100,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(200,kernel_initializer = 'uniform',  activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [43]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=2000, batch_size=128, verbose=1)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.5506 - accuracy: 0.7383
Epoch 2/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4562 - accuracy: 0.7757
Epoch 3/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4458 - accuracy: 0.7893
Epoch 4/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4362 - accuracy: 0.7977
Epoch 5/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4275 - accuracy: 0.8009
Epoch 6/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4234 - accuracy: 0.8042
Epoch 7/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.8080
Epoch 8/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4188 - accuracy: 0.8064
Epoch 9/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4137 - accuracy: 0.8124
Epoch 10/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.4077 - accuracy: 0.8159

54/54 [==============================] - 0s 1ms/step - loss: 0.2958 - accuracy: 0.8702
Epoch 83/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2977 - accuracy: 0.8676
Epoch 84/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2969 - accuracy: 0.8685
Epoch 85/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2917 - accuracy: 0.8727
Epoch 86/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2890 - accuracy: 0.8717
Epoch 87/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2865 - accuracy: 0.8755
Epoch 88/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2887 - accuracy: 0.8708
Epoch 89/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2883 - accuracy: 0.8705
Epoch 90/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2863 - accuracy: 0.8748
Epoch 91/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2862 - accuracy: 0.8764
Epoc

54/54 [==============================] - 0s 2ms/step - loss: 0.2463 - accuracy: 0.8993
Epoch 163/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2425 - accuracy: 0.8993
Epoch 164/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2469 - accuracy: 0.8957
Epoch 165/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2459 - accuracy: 0.8963
Epoch 166/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2452 - accuracy: 0.8957
Epoch 167/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2413 - accuracy: 0.8986
Epoch 168/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2428 - accuracy: 0.8973
Epoch 169/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2388 - accuracy: 0.8995
Epoch 170/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2424 - accuracy: 0.8982
Epoch 171/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.2432 - accuracy: 0.

54/54 [==============================] - 0s 1ms/step - loss: 0.2193 - accuracy: 0.9075
Epoch 243/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2091 - accuracy: 0.9110
Epoch 244/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2124 - accuracy: 0.9112
Epoch 245/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2115 - accuracy: 0.9110
Epoch 246/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2097 - accuracy: 0.9134
Epoch 247/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2149 - accuracy: 0.9119
Epoch 248/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2154 - accuracy: 0.9061
Epoch 249/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2107 - accuracy: 0.9116
Epoch 250/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2160 - accuracy: 0.9094
Epoch 251/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.2035 - accuracy: 0.

54/54 [==============================] - 0s 1ms/step - loss: 0.1850 - accuracy: 0.9229
Epoch 323/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1884 - accuracy: 0.9235
Epoch 324/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1903 - accuracy: 0.9214
Epoch 325/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1939 - accuracy: 0.9184
Epoch 326/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1988 - accuracy: 0.9176
Epoch 327/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1851 - accuracy: 0.9204
Epoch 328/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1832 - accuracy: 0.9252
Epoch 329/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1902 - accuracy: 0.9211
Epoch 330/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1897 - accuracy: 0.9219
Epoch 331/2000
54/54 [==============================] - 0s 1ms/step - loss: 0.1829 - accuracy: 0.

...

54/54 [==============================] - 0s 2ms/step - loss: 0.1087 - accuracy: 0.9563
Epoch 1627/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1050 - accuracy: 0.9582
Epoch 1628/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1075 - accuracy: 0.9538
Epoch 1629/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1074 - accuracy: 0.9571
Epoch 1630/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1048 - accuracy: 0.9579
Epoch 1631/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1009 - accuracy: 0.9571
Epoch 1632/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1064 - accuracy: 0.9561
Epoch 1633/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1130 - accuracy: 0.9545
Epoch 1634/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1113 - accuracy: 0.9542
Epoch 1635/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1158 - acc

54/54 [==============================] - 0s 2ms/step - loss: 0.1042 - accuracy: 0.9583
Epoch 1706/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1017 - accuracy: 0.9576
Epoch 1707/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1021 - accuracy: 0.9567
Epoch 1708/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1076 - accuracy: 0.9557
Epoch 1709/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1045 - accuracy: 0.9554
Epoch 1710/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1077 - accuracy: 0.9541
Epoch 1711/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1057 - accuracy: 0.9566
Epoch 1712/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1029 - accuracy: 0.9561
Epoch 1713/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1059 - accuracy: 0.9573
Epoch 1714/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1101 - acc

54/54 [==============================] - 0s 2ms/step - loss: 0.0982 - accuracy: 0.9582
Epoch 1785/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1012 - accuracy: 0.9564
Epoch 1786/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1106 - accuracy: 0.9558
Epoch 1787/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1088 - accuracy: 0.9542
Epoch 1788/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1089 - accuracy: 0.9561
Epoch 1789/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1210 - accuracy: 0.9544
Epoch 1790/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1206 - accuracy: 0.9523
Epoch 1791/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1117 - accuracy: 0.9557
Epoch 1792/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1074 - accuracy: 0.9571
Epoch 1793/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1075 - acc

54/54 [==============================] - 0s 2ms/step - loss: 0.1048 - accuracy: 0.9561
Epoch 1864/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1005 - accuracy: 0.9563
Epoch 1865/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1083 - accuracy: 0.9560
Epoch 1866/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1065 - accuracy: 0.9561
Epoch 1867/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0994 - accuracy: 0.9596
Epoch 1868/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0987 - accuracy: 0.9585
Epoch 1869/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0998 - accuracy: 0.9599
Epoch 1870/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0999 - accuracy: 0.9596
Epoch 1871/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0986 - accuracy: 0.9590
Epoch 1872/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1029 - acc

54/54 [==============================] - 0s 2ms/step - loss: 0.1056 - accuracy: 0.9545
Epoch 1943/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1047 - accuracy: 0.9579
Epoch 1944/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1099 - accuracy: 0.9548
Epoch 1945/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1029 - accuracy: 0.9582
Epoch 1946/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1174 - accuracy: 0.9525
Epoch 1947/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1034 - accuracy: 0.9574
Epoch 1948/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.0972 - accuracy: 0.9608
Epoch 1949/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1011 - accuracy: 0.9585
Epoch 1950/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1020 - accuracy: 0.9576
Epoch 1951/2000
54/54 [==============================] - 0s 2ms/step - loss: 0.1040 - acc

In [44]:
print(results)

[0.89174956 0.89467525 0.89584553 0.90403748 0.89578456]


In [45]:
# estimator.fit(X_train, y_train)
# predictions = estimator.predict(X_test)
#print(predictions)

In [46]:
# print(encoder.inverse_transform(predictions))

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

Epoch 1/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.5609 - accuracy: 0.7266
Epoch 2/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.4662 - accuracy: 0.7701
Epoch 3/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.4539 - accuracy: 0.7861
Epoch 4/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.4491 - accuracy: 0.7888
Epoch 5/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.4415 - accuracy: 0.7906
Epoch 6/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.4355 - accuracy: 0.7977
Epoch 7/2000
47/47 [==============================] - 0s 3ms/step - loss: 0.4341 - accuracy: 0.7963
Epoch 8/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.8015
Epoch 9/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.7980
Epoch 10/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.8025

47/47 [==============================] - 0s 1ms/step - loss: 0.3065 - accuracy: 0.8664
Epoch 83/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2996 - accuracy: 0.8711
Epoch 84/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.3070 - accuracy: 0.8619
Epoch 85/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.3001 - accuracy: 0.8647
Epoch 86/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.3038 - accuracy: 0.8642
Epoch 87/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.3008 - accuracy: 0.8671
Epoch 88/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2999 - accuracy: 0.8699
Epoch 89/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.3069 - accuracy: 0.8622
Epoch 90/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2977 - accuracy: 0.8656
Epoch 91/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2973 - accuracy: 0.8642
Epoc

47/47 [==============================] - 0s 1ms/step - loss: 0.2512 - accuracy: 0.8913
Epoch 163/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2514 - accuracy: 0.8923
Epoch 164/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2511 - accuracy: 0.8948
Epoch 165/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2538 - accuracy: 0.8943
Epoch 166/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2539 - accuracy: 0.8950
Epoch 167/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2486 - accuracy: 0.8923
Epoch 168/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2482 - accuracy: 0.8931
Epoch 169/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2469 - accuracy: 0.8962
Epoch 170/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2577 - accuracy: 0.8901
Epoch 171/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2473 - accuracy: 0.

47/47 [==============================] - 0s 1ms/step - loss: 0.2195 - accuracy: 0.9060
Epoch 243/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2174 - accuracy: 0.9064
Epoch 244/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2277 - accuracy: 0.9077
Epoch 245/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2311 - accuracy: 0.9025
Epoch 246/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2210 - accuracy: 0.9062
Epoch 247/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2159 - accuracy: 0.9100
Epoch 248/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2230 - accuracy: 0.9013
Epoch 249/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2150 - accuracy: 0.9090
Epoch 250/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2106 - accuracy: 0.9142
Epoch 251/2000
47/47 [==============================] - 0s 1ms/step - loss: 0.2149 - accuracy: 0.

47/47 [==============================] - 0s 2ms/step - loss: 0.1985 - accuracy: 0.9176
Epoch 323/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1963 - accuracy: 0.9191
Epoch 324/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1872 - accuracy: 0.9236
Epoch 325/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1903 - accuracy: 0.9214
Epoch 326/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1908 - accuracy: 0.9226
Epoch 327/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.9206
Epoch 328/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1876 - accuracy: 0.9214
Epoch 329/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.9227
Epoch 330/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1878 - accuracy: 0.9236
Epoch 331/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1894 - accuracy: 0.

...

47/47 [==============================] - 0s 2ms/step - loss: 0.1075 - accuracy: 0.9559
Epoch 1675/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1041 - accuracy: 0.9557
Epoch 1676/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1031 - accuracy: 0.9599
Epoch 1677/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1033 - accuracy: 0.9569
Epoch 1678/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1057 - accuracy: 0.9562
Epoch 1679/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1029 - accuracy: 0.9577
Epoch 1680/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1090 - accuracy: 0.9540
Epoch 1681/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1056 - accuracy: 0.9543
Epoch 1682/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1008 - accuracy: 0.9580
Epoch 1683/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1011 - acc

47/47 [==============================] - 0s 2ms/step - loss: 0.1064 - accuracy: 0.9559
Epoch 1754/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1111 - accuracy: 0.9522
Epoch 1755/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1181 - accuracy: 0.9508
Epoch 1756/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1107 - accuracy: 0.9543
Epoch 1757/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1043 - accuracy: 0.9575
Epoch 1758/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1137 - accuracy: 0.9550
Epoch 1759/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1146 - accuracy: 0.9528
Epoch 1760/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1050 - accuracy: 0.9574
Epoch 1761/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1072 - accuracy: 0.9545
Epoch 1762/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1008 - acc

47/47 [==============================] - 0s 2ms/step - loss: 0.1066 - accuracy: 0.9567
Epoch 1833/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1036 - accuracy: 0.9570
Epoch 1834/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1037 - accuracy: 0.9565
Epoch 1835/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0999 - accuracy: 0.9574
Epoch 1836/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0982 - accuracy: 0.9600
Epoch 1837/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1006 - accuracy: 0.9589
Epoch 1838/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0978 - accuracy: 0.9614
Epoch 1839/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 0.9609
Epoch 1840/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1124 - accuracy: 0.9535
Epoch 1841/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1063 - acc

47/47 [==============================] - 0s 2ms/step - loss: 0.0977 - accuracy: 0.9607
Epoch 1912/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0964 - accuracy: 0.9609
Epoch 1913/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0960 - accuracy: 0.9600
Epoch 1914/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0978 - accuracy: 0.9582
Epoch 1915/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0976 - accuracy: 0.9624
Epoch 1916/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1045 - accuracy: 0.9562
Epoch 1917/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1037 - accuracy: 0.9542
Epoch 1918/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0986 - accuracy: 0.9599
Epoch 1919/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1056 - accuracy: 0.9547
Epoch 1920/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1037 - acc

47/47 [==============================] - 0s 2ms/step - loss: 0.0953 - accuracy: 0.9614
Epoch 1991/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0946 - accuracy: 0.9594
Epoch 1992/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0930 - accuracy: 0.9615
Epoch 1993/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0994 - accuracy: 0.9587
Epoch 1994/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1015 - accuracy: 0.9585
Epoch 1995/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.9585
Epoch 1996/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1109 - accuracy: 0.9565
Epoch 1997/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.1064 - accuracy: 0.9577
Epoch 1998/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0969 - accuracy: 0.9589
Epoch 1999/2000
47/47 [==============================] - 0s 2ms/step - loss: 0.0947 - acc

In [50]:
print(predictions[0])
# print(encoder.inverse_transform(predictions))

[0]


In [51]:
Y_test[0]

0

In [52]:
import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[3]
# 2

ValueError: `axis` must be fewer than the number of dimensions (1)

In [53]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, predictions)
cm

array([[1115,  159],
       [ 120, 1170]])

In [55]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      1274
           1       0.88      0.91      0.89      1290

    accuracy                           0.89      2564
   macro avg       0.89      0.89      0.89      2564
weighted avg       0.89      0.89      0.89      2564



In [57]:
accuracy_score(Y_test, predictions)
print('Accuracy is ' + str(round(accuracy_score(Y_test, predictions),2)))

Accuracy is 0.89


In [58]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [59]:
cmn= np.around(cm / cm.astype(np.float).sum(axis=1, keepdims=True),2) 

In [60]:
cmn

array([[0.88, 0.12],
       [0.09, 0.91]])